In [38]:
import pandas as pd
import numpy as np

In [39]:
def calculaPorDia(x):    
    #print(x[3]/x[6].days)
    #print(x[3],'',x[7])
    return x[3]/x[7].days


In [40]:
hotels = []
#filename = 'hotels-in-Veneza.txt'
#filename = 'hotels-in-Zurich.txt'
filename = 'hotels-in-Munich.txt'
#filename = 'hotels-in-Lucerna.txt'

with open(filename) as f:
    for line in f:
        if len(line)>10:
            hotels.append([str(n.replace('\xa0','').replace('"','').replace('[','').replace(']','')) for n in line.strip().split(';')])
            #print(line)

In [41]:
import datetime
hotelDict = []
for i in hotels:
    #print(i[0][3:5])
    dtIni = datetime.datetime(int(i[0][6:10]), int(i[0][3:5]), int(i[0][0:2]), 0, 0, 0)
    dtEnd = datetime.datetime(int(i[1][6:10]), int(i[1][3:5]), int(i[1][0:2]), 0, 0, 0)
    hotelDict.append({'de':dtIni,'ate':dtEnd,'nome':i[2],'preco':i[3],'nota':i[4],'distancia':i[5],'url':i[6]})

In [42]:
df = pd.DataFrame(hotelDict)
df.drop_duplicates(inplace = True) 
df = df[df['preco']>'-1']
df['preco'].replace(regex=True,inplace=True,to_replace=r'\D',value=r'')
df['preco'] = pd.to_numeric(df['preco'])
df['dias'] = (df['ate']-df['de']).replace(regex=True,to_replace=r'\D',value=r'')
df["precoPorNoite"] = df.apply(calculaPorDia,axis=1)
display(df.describe())


,preco,dias,precoPorNoite
count,425.000000,425,425.000000
mean,2837.091765,4 days 00:00:00,709.272941
std,1052.008456,0 days 00:00:00,263.002114
min,536.000000,4 days 00:00:00,134.000000
25%,2312.000000,4 days 00:00:00,578.000000
50%,2763.000000,4 days 00:00:00,690.750000
75%,3336.000000,4 days 00:00:00,834.000000
max,5686.000000,4 days 00:00:00,1421.500000


In [43]:

datetimeFormat = '%Y-%m-%d %H:%M:%S.%f'


#df['diasteste'] = datetime.datetime.strptime(df['ate'],datetimeFormat) - datetime.datetime.strptime(df['de'],datetimeFormat)
df['dias'] = (df['ate']-df['de']).replace(regex=True,to_replace=r'\D',value=r'')
df["nota"] = pd.to_numeric(df["nota"])
df['distanciaMedida'] = pd.np.where(df.distancia.str.contains("km"),"km","m")
df["distanciaNumerica"] = pd.to_numeric(df['distancia'].replace(regex=True,to_replace=r'\D',value=r''))#, downcast='float')
df["distanciaMetros"] = pd.np.where(df.distancia.str.contains("km") & df.distancia.str.contains("."),df.distanciaNumerica*1000,df["distanciaNumerica"])
df["distanciaMetros"] = pd.np.where(df.distanciaMetros > 9000,(df.distanciaNumerica/10),df["distanciaNumerica"])


df["distanciaMetros"] = df["distanciaMetros"].apply(lambda x: x*1000 if x < 100 else x)
#df["precoPorNoite"] = df.apply(calculaPorDia,axis=1)


df = df[df['precoPorNoite']<1000]
df = df[df['nota']>7.5]
df = df[df['distanciaMetros']<2000]
df = df[~df["nome"].str.contains('Hostel')]

del df['distanciaMedida']
del df['distanciaNumerica']


df.drop_duplicates(inplace = True) 
display(df.describe())
display(df)




,preco,nota,dias,precoPorNoite,distanciaMetros
count,93.000000,93.000000,93,93.000000,93.000000
mean,2754.526882,8.201075,4 days 00:00:00,688.631720,1152.150538
std,813.671692,0.337347,0 days 00:00:00,203.417923,390.506498
min,1111.000000,7.800000,4 days 00:00:00,277.750000,600.000000
25%,2763.000000,7.900000,4 days 00:00:00,690.750000,950.000000
50%,2881.000000,8.100000,4 days 00:00:00,720.250000,1100.000000
75%,3336.000000,8.600000,4 days 00:00:00,834.000000,1200.000000
max,3945.000000,8.700000,4 days 00:00:00,986.250000,1900.000000


,de,ate,nome,preco,nota,distancia,url,dias,precoPorNoite,distanciaMetros
0,2020-03-05,2020-03-09,AWA Hotel,3258,8.1,1.1 km from centre,www.booking.com/hotel/de/devillemuenchen.en-gb...,4 days,814.50,1100.0
17,2020-03-05,2020-03-09,Euro Youth Hotel Munich,1111,8.7,1.2 km from centre,www.booking.com/hotel/de/euro-youth.en-gb.html...,4 days,277.75,1200.0
26,2020-03-05,2020-03-09,Hotel Stachus,2763,7.9,950 m from centre,www.booking.com/hotel/de/stachus-munchen.en-gb...,4 days,690.75,950.0
33,2020-03-05,2020-03-09,Euro Youth Hotel Munich,1111,8.7,1.2 km from centre,www.booking.com/hotel/de/euro-youth.en-gb.html...,4 days,277.75,1200.0
39,2020-03-05,2020-03-09,Hotel Stachus,2763,7.9,950 m from centre,www.booking.com/hotel/de/stachus-munchen.en-gb...,4 days,690.75,950.0
48,2020-03-05,2020-03-09,Euro Youth Hotel Munich,1111,8.7,1.2 km from centre,www.booking.com/hotel/de/euro-youth.en-gb.html...,4 days,277.75,1200.0
53,2020-03-05,2020-03-09,Hotel Senator Mxc3xcnchen,2881,8.2,1.9 km from centre,www.booking.com/hotel/de/senator-munchen.en-gb...,4 days,720.25,1900.0
54,2020-03-05,2020-03-09,Hotel Luitpold,2926,8.0,1.1 km from centre,www.booking.com/hotel/de/luitpoldmuenchen.en-g...,4 days,731.50,1100.0
56,2020-03-05,2020-03-09,Hotel MIO y AMANO,3590,8.6,600 m from centre,www.booking.com/hotel/de/mio-by-amano.en-gb.ht...,4 days,897.50,600.0
57,2020-03-05,2020-03-09,City-Hotel Mxc3xcnchen,3336,7.8,1.1 km from centre,www.booking.com/hotel/de/city-munchen.en-gb.ht...,4 days,834.00,1100.0


In [44]:
import plotly.express as px
#fig = px.scatter(df, x="precoPorNoite", y="nota",color="nome")
#fig = px.scatter(df, x="preco", y="nota",color="nome")
#fig.show()

In [45]:
import plotly.express as px
fig = px.scatter(df, x="precoPorNoite", y="nota", size="distanciaMetros", color="nome",hover_name="nome",log_x=True)
fig.show()
fig.write_html(filename+".html") 

In [46]:
df['link'] ='<a href="http://'+df['url']+'">Clique</a>'

df= df.drop_duplicates(subset=['nome', 'preco'])
dfa = df.copy()
del dfa['url']
del dfa['de']
del dfa['ate']
del dfa['dias']
del dfa['distanciaMetros']
dfa = dfa.to_html()
print('de: ',df['de'].max())
print('até: ',df['ate'].max())
#display(dfa)
#display(df.drop_duplicates().sort_values(by=['precoPorNoite'],ascending=False))
from IPython.core.display import display, HTML
display(HTML(dfa))

de:  2020-03-05 00:00:00
até:  2020-03-09 00:00:00


,nome,preco,nota,distancia,precoPorNoite,link
0,AWA Hotel,3258,8.1,1.1 km from centre,814.50,"<a href=""http://www.booking.com/hotel/de/devillemuenchen.en-gb.html?aid=304142&label=gen173nr-1BCAEoggI46AdIM1gEaJMBiAEBmAEJuAEXyAEM2AEB6AEBiAIBqAID&sid=0264f227627e990b07cba1276bca7042&all_sr_blocks=6096608_196578731_0_0_0%2C6096608_196578731_0_0_0&checkin=2020-03-05&checkout=2020-03-09&dest_id=-1829149&dest_type=city&group_adults=1&group_children=0&hapos=4&highlighted_blocks=6096608_196578731_0_0_0%2C6096608_196578731_0_0_0&hpos=1&nflt=roomfacility%3D38%3B&no_rooms=2&req_adults=1&req_children=0&room1=A&room2=A&sr_order=popularity&sr_pri_blocks=6096608_196578731_0_0_0__36072%2C6096608_196578731_0_0_0__36072&srepoch=1578179411&srpvid=e573a2e9213b006e&ucfs=1&from=searchresults"">Clique</a>"
17,Euro Youth Hotel Munich,1111,8.7,1.2 km from centre,277.75,"<a href=""http://www.booking.com/hotel/de/euro-youth.en-gb.html?aid=304142&label=gen173nr-1BCAEoggI46AdIM1gEaJMBiAEBmAEJuAEXyAEM2AEB6AEBiAIBqAID&sid=7f7617d77bc8869baa397004c985b585&all_sr_blocks=7946021_138720173_0_2_0%2C7946021_138720173_0_2_0&checkin=2020-03-05&checkout=2020-03-09&dest_id=-1829149&dest_type=city&group_adults=1&group_children=0&hapos=19&highlighted_blocks=7946021_138720173_0_2_0%2C7946021_138720173_0_2_0&hpos=6&nflt=roomfacility%3D38%3B&no_rooms=2&req_adults=1&req_children=0&room1=A&room2=A&sr_order=popularity&sr_pri_blocks=7946021_138720173_0_2_0__12300%2C7946021_138720173_0_2_0__12300&srepoch=1578179412&srpvid=ada1a2e9cf090188&ucfs=1&from=searchresults"">Clique</a>"
26,Hotel Stachus,2763,7.9,950 m from centre,690.75,"<a href=""http://www.booking.com/hotel/de/stachus-munchen.en-gb.html?aid=304142&label=gen173nr-1BCAEoggI46AdIM1gEaJMBiAEBmAEJuAEXyAEM2AEB6AEBiAIBqAID&sid=7f7617d77bc8869baa397004c985b585&all_sr_blocks=7440101_121581685_0_2_0%2C7440105_121581685_0_2_0&checkin=2020-03-05&checkout=2020-03-09&dest_id=-1829149&dest_type=city&group_adults=1&group_children=0&hapos=22&highlighted_blocks=7440101_121581685_0_2_0%2C7440105_121581685_0_2_0&hpos=9&nflt=roomfacility%3D38%3B&no_rooms=2&req_adults=1&req_children=0&room1=A&room2=A&sr_order=popularity&sr_pri_blocks=7440101_121581685_0_2_0__28600%2C7440105_121581685_0_2_0__32600&srepoch=1578179412&srpvid=ada1a2e9cf090188&ucfs=1&from=searchresults"">Clique</a>"
53,Hotel Senator Mxc3xcnchen,2881,8.2,1.9 km from centre,720.25,"<a href=""http://www.booking.com/hotel/de/senator-munchen.en-gb.html?aid=304142&label=gen173nr-1BCAEoggI46AdIM1gEaJMBiAEBmAEJuAEXyAEM2AEB6AEBiAIBqAID&sid=7f7617d77bc8869baa397004c985b585&all_sr_blocks=25878101_199270029_0_1_0%2C25878101_199270029_0_1_0&checkin=2020-03-05&checkout=2020-03-09&dest_id=-1829149&dest_type=city&group_adults=1&group_children=0&hapos=31&highlighted_blocks=25878101_199270029_0_1_0%2C25878101_199270029_0_1_0&hpos=18&nflt=roomfacility%3D38%3B&no_rooms=2&req_adults=1&req_children=0&room1=A&room2=A&sr_order=popularity&sr_pri_blocks=25878101_199270029_0_1_0__31900%2C25878101_199270029_0_1_0__31900&srepoch=1578179412&srpvid=ada1a2e9cf090188&ucfs=1&from=searchresults"">Clique</a>"
54,Hotel Luitpold,2926,8.0,1.1 km from centre,731.50,"<a href=""http://www.booking.com/hotel/de/luitpoldmuenchen.en-gb.html?aid=304142&label=gen173nr-1BCAEoggI46AdIM1gEaJMBiAEBmAEJuAEXyAEM2AEB6AEBiAIBqAID&sid=7f7617d77bc8869baa397004c985b585&all_sr_blocks=6151801_200884162_0_1_0%2C6151801_200884162_0_1_0&checkin=2020-03-05&checkout=2020-03-09&dest_id=-1829149&dest_type=city&group_adults=1&group_children=0&hapos=32&highlighted_blocks=6151801_200884162_0_1_0%2C6151801_200884162_0_1_0&hpos=19&nflt=roomfacility%3D38%3B&no_rooms=2&req_adults=1&req_children=0&room1=A&room2=A&sr_order=popularity&sr_pri_blocks=6151801_200884162_0_1_0__32400%2C6151801_200884162_0_1_0__32400&srepoch=1578179412&srpvid=ada1a2e9cf090188&ucfs=1&from=searchresults"">Clique</a>"
56,Hotel MIO y AMANO,3590,8.6,600 m from centre,897.50,"<a href=""http://www.booking.com/hotel/de/mio-by-amano.en-gb.html?aid=304142&label=gen173nr-1BCAEoggI46AdIM1